In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import gc
import cudf
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from cuml.ensemble import RandomForestClassifier as cuRFC
from cuml.metrics import roc_auc_score
from sklearn.model_selection import KFold

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = cudf.read_csv('/kaggle/input/tabular-playground-series-sep-2021/train.csv')
test = cudf.read_csv('/kaggle/input/tabular-playground-series-sep-2021/test.csv')
sample_soln_df = cudf.read_csv('/kaggle/input/tabular-playground-series-sep-2021/sample_solution.csv')

In [ ]:
# Getting the features and target column:
FEATURES = train.columns[:-1]
TARGET = train.columns[-1]

In [ ]:
# Creating new features:
train["Missing_Value"] = train[FEATURES].isna().sum(axis=1) 
train["Missing_Value_cat"] = train["Missing_Value"].map(lambda x: 0 if x==0 else 1)
test["Missing_Value"] = test[FEATURES].isna().sum(axis=1) 
test["Missing_Value_cat"] = test["Missing_Value"].map(lambda x: 0 if x==0 else 1)

In [ ]:
# Filling in the missing values with avg value of the column:
for col in FEATURES:
    avg_val = train[col].median()
    train[col].fillna(avg_val, inplace=True)
    test[col].fillna(avg_val, inplace=True)

# Extending the list of features:
FEATURES = list(FEATURES)
FEATURES.extend(['Missing_Value','Missing_Value_cat'])

In [ ]:
test_preds = 0
i = 1
train_oof = np.zeros((train.shape[0],1))
train_oof = pd.DataFrame(train_oof, columns=['preds'])
kf = KFold(n_splits=5)
for train_index, test_index in kf.split(train[FEATURES]):
    print(i)
    clf = cuRFC()
    trn_X, trn_y = train.loc[train_index, FEATURES], train.loc[train_index, TARGET]
    val_X, val_y = train.loc[test_index, FEATURES], train.loc[test_index, TARGET]
    clf.fit(trn_X.astype(np.float32),trn_y.astype(np.int32))
    oof_proba = (clf.predict_proba(val_X.astype(np.float32))[1]).to_pandas().values
    train_oof.loc[test_index,'preds'] = oof_proba
    tmp = (clf.predict_proba(test[FEATURES].astype(np.float32))[1]).to_pandas().values
    test_preds += tmp/5
    i +=1 
    gc.collect()
    
print("The score for OOF is:")
roc_auc_score(train['claim'], train_oof['preds'])

In [ ]:
sample_soln_df['claim'] = test_preds
sample_soln_df.to_csv('submission.csv', index=False)